# How to deploy ADK Apps to Agent Engine

[Documentation link 🔗](https://google.github.io/adk-docs/deploy/agent-engine/)

## 1. Set the local environment variables
These will be used in the Agent Engine deployment

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()  # take environment variables

True

## 2. Set up the `AdkApp` Convenience wrapper for ADK Agents -> Agent Engine

Note the injection of the environment variables.

In [2]:
from vertexai.preview.reasoning_engines import AdkApp
from google.adk.artifacts import GcsArtifactService
# from google.adk.sessions import VertexAiSessionService

from obelisk_recontext_agent import agent

env_vars = {}

env_vars["GOOGLE_GENAI_USE_VERTEXAI"] = os.getenv("GOOGLE_GENAI_USE_VERTEXAI")
env_vars["BUCKET"] = os.getenv("BUCKET", "gs://default-bucket")


def artifact_service_builder():
    return GcsArtifactService(bucket_name=env_vars["BUCKET"].split("gs://")[1])


my_agent = AdkApp(
    agent=agent.root_agent,
    enable_tracing=True,
    env_vars=env_vars,
    artifact_service_builder=artifact_service_builder,
)

# 3. Test the Agent Engine locally

In [4]:
session = my_agent.create_session(user_id="jwortz", session_id="12345")
session

Session(id='12345', app_name='default-app-name', user_id='jwortz', state={}, events=[], last_update_time=1752780544.3658361)

In [5]:
my_agent.list_sessions(user_id="jwortz")

ListSessionsResponse(sessions=[Session(id='12345', app_name='default-app-name', user_id='jwortz', state={}, events=[], last_update_time=1752780544.3658361)])

In [6]:
from pprint import pprint


def stream_agent(agent, prompt: str, session) -> None | Exception:
    try:
        session_id = session['id']
    # local prefers attributes vs. remote prefers dict keys
    except TypeError:
        session_id = session.id
    except Exception as e:
        return f"Session Object not valid: {e}"
        
    for event in agent.stream_query(
        user_id="jwortz",
        session_id=session_id,
        message=prompt,
    ):
        pprint(event)

In [7]:
first_prompt = f"Hello"
stream_agent(my_agent, first_prompt, session)

{'actions': {'artifact_delta': {},
             'requested_auth_configs': {},
             'state_delta': {}},
 'author': 'product_recontextualiztion_agent',
 'content': {'parts': [{'text': "Hello! I'm here to help you create compelling "
                                'visual assets.\n'
                                '\n'
                                'To start, please upload the product images '
                                "you'd like to recontextualize. Once I have "
                                'those, tell me how you envision your product '
                                'being presented. For example, "Place these '
                                'headphones on a desk in a futuristic sci-fi '
                                'lab," or "Show this watch in a serene, '
                                'natural outdoor setting."'}],
             'role': 'model'},
 'id': 'aflTunee',
 'invocation_id': 'e-f68e402d-3f58-4bcf-b242-efc9d2bc8de2',
 'timestamp': 1752780545.685166,


## 4. Deploy to Vertex AI Agent Engine
To call and manage agents in production, deploy the agent to Vertex AI Agent Engine.

**Important - run `poetry build` to package the agent**

In [3]:
! poetry build --format=wheel --output=deployment

Building obelisk_recontext_agent (0.1.0)
  - Building wheel
  - Built obelisk_recontext_agent-0.1.0-py3-none-any.whl


#### Initialize the Vertex client, then create a `remote_agent` that is deployed to Vertex

This also takes the packaged agent code. This is required for more complex agents that have nested dependencies and require packaging

Also, before running - be sure to give Secret Manager access to the Agent Engine service account. This can simply be done by running `. setup_ae_sm_access.sh`.

The code can also be ran as follows (relative to repo root):

```bash
source trends_and_insights_agent/.env

export RE_SA="service-${GOOGLE_CLOUD_PROJECT_NUMBER}@gcp-sa-aiplatform-re.iam.gserviceaccount.com"
gcloud secrets add-iam-policy-binding "projects/$GOOGLE_CLOUD_PROJECT/secrets/$YT_SECRET_MNGR_NAME" \
  --member="serviceAccount:$RE_SA" \
  --role="roles/secretmanager.secretAccessor"
```

In [ ]:
# from vertexai import agent_engines # optional cleanup

# for agent in agent_engines.list():
#     agent.delete(force=True)

In [ ]:
import vertexai
from vertexai import agent_engines
import os

GOOGLE_CLOUD_PROJECT = os.getenv("GOOGLE_CLOUD_PROJECT")
BUCKET = os.getenv("BUCKET")

vertexai.init(
    project=GOOGLE_CLOUD_PROJECT,
    location="us-central1",
    staging_bucket=BUCKET,
)

remote_agent = agent_engines.create(
    agent_engine=my_agent,
    display_name="prism-agent",
    description="You are an agent that takes product images and recontextualizes them in different settings, followed up with creating some video concepts from the generated images.",
    requirements=[
        "deployment/obelisk_recontext_agent-0.1.0-py3-none-any.whl",
    ],
    extra_packages=[
        "deployment/obelisk_recontext_agent-0.1.0-py3-none-any.whl",
    ],
    env_vars=env_vars,
)

Identified the following requirements: {'cloudpickle': '3.1.1', 'pydantic': '2.11.7', 'google-cloud-aiplatform': '1.100.0'}
Failed to parse constraint: deployment/obelisk_recontext_agent-0.1.0-py3-none-any.whl. Exception: Expected end or semicolon (after name and no valid version specifier)
    deployment/obelisk_recontext_agent-0.1.0-py3-none-any.whl
              ^
The following requirements are missing: {'cloudpickle', 'pydantic', 'google-cloud-aiplatform'}
The following requirements are appended: {'pydantic==2.11.7', 'cloudpickle==3.1.1'}
The final list of requirements: ['deployment/obelisk_recontext_agent-0.1.0-py3-none-any.whl', 'pydantic==2.11.7', 'cloudpickle==3.1.1']
Using bucket prism-research-25
Wrote to gs://prism-research-25/agent_engine/agent_engine.pkl
Writing to gs://prism-research-25/agent_engine/requirements.txt
Creating in-memory tarfile of extra_packages
Writing to gs://prism-research-25/agent_engine/dependencies.tar.gz
Creating AgentEngine
Create AgentEngine backin

### Save the agent engine resource name

This can be used by the command to deploy it to Agentspace

```bash
agent_engine = vertexai.agent_engines.get('projects/679926387543/locations/us-central1/reasoningEngines/1093257605637210112')
```

## 5. Try it remotely

In [18]:
online_session = remote_agent.create_session(user_id="jwortz")
online_session

{'lastUpdateTime': 1752610947.892835,
 'events': [],
 'userId': 'jwortz',
 'state': {},
 'appName': '1796681164623183872',
 'id': '7914058747092140032'}

#### Same idea above applies to remote agents

In [19]:
first_prompt = f"Hello"

stream_agent(remote_agent, first_prompt, online_session)

{'actions': {'artifact_delta': {},
             'requested_auth_configs': {},
             'state_delta': {'_state_init': True,
                             'artifact_keys': {'image_creatives': {},
                                               'video_creatives': {}},
                             'campaign_guide': {'brand': 'Paul Reed Smith '
                                                         '(PRS)',
                                                'campaign_name': 'Marketing '
                                                                 'Campaign '
                                                                 'Guide: PRS '
                                                                 'SE CE24 '
                                                                 'Guitar',
                                                'campaign_objectives': ['Increase '
                                                                        'Brand '
                                       

In [37]:
second_prompt = f"hi" # select an arbitrary trend (#2 on the list for that day)
stream_agent(remote_agent, second_prompt, online_session)

{'actions': {'artifact_delta': {},
             'requested_auth_configs': {},
             'state_delta': {}},
 'author': 'trends_and_insights_agent',
 'content': {'parts': [{'text': 'Please choose a trending topic from the list '
                                'above.'}],
             'role': 'model'},
 'id': 'vXm1ERH4',
 'invocation_id': 'e-6bc3a0a6-b11c-427c-b515-484c5f737e28',
 'timestamp': 1752539837.354329,
 'usage_metadata': {'candidates_token_count': 10,
                    'candidates_tokens_details': [{'modality': 'TEXT',
                                                   'token_count': 10}],
                    'prompt_token_count': 1702,
                    'prompt_tokens_details': [{'modality': 'TEXT',
                                               'token_count': 1702}],
                    'thoughts_token_count': 74,
                    'total_token_count': 1786,
                    'traffic_type': 'ON_DEMAND'}}


# 6. Optional Cleanup

In [ ]:
remote_agent.delete(force=True)